In [ ]:
#!/usr/bin/env python3
# mqtt_subscriber.py

import json
import time
import threading
import pytz
import socket
import paho.mqtt.client as mqtt
from datetime import datetime
from jetbot import Robot        # 로봇 제어 모듈
from SCSCtrl import TTLServo    # 서보 제어 모듈

# --- 로컬 IP 획득 함수 ---
def get_local_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(("8.8.8.8", 80))
        return s.getsockname()[0]
    except Exception:
        return None
    finally:
        s.close()

LOCAL_IP = get_local_ip()

# --- 전역 변수 설정 ---
korea_tz      = pytz.timezone("Asia/Seoul")
BROKER_ADDR   = "172.20.10.6"
BROKER_PORT   = 1883
COMMAND_TOPIC = "AGV/command"

robot = Robot()

# 제어 이벤트
auto_event    = threading.Event()
manual_event  = threading.Event()
exit_event    = threading.Event()
manual_cmd    = None

def motion_loop():
    """
    auto_event 또는 manual_event에 따라
    100ms 간격으로 연속 제어를 수행.
    """
    while not exit_event.is_set():
        if auto_event.is_set():
            # 자동 모드: 일정 속도로 전진
            robot.forward(0.8)
        elif manual_event.is_set():
            # 수동 모드: 마지막 수신된 명령 실행
            if manual_cmd == "go":
                robot.forward(0.8)
            elif manual_cmd == "left":
                robot.left(0.6)
            elif manual_cmd == "right":
                robot.right(0.6)
            elif manual_cmd == "back":
                robot.backward(0.8)
        else:
            # 그 외: 정지
            robot.stop()
        time.sleep(0.1)
    # 종료 시 반드시 정지
    robot.stop()

# --- MQTT 콜백 함수 ---
def on_connect(client, userdata, flags, rc):
    print(f"[{datetime.now(korea_tz)}] MQTT 연결 {'성공' if rc==0 else '실패 코드='+str(rc)}")
    if rc == 0:
        client.subscribe(COMMAND_TOPIC, qos=1)
        print(f"[{datetime.now(korea_tz)}] 구독 시작 → {COMMAND_TOPIC}")

def on_publish(client, userdata, mid):
    print(f"[{datetime.now(korea_tz)}] publish 완료, mid={mid}")

def on_message(client, userdata, msg):
    global manual_cmd
    try:
        message = json.loads(msg.payload.decode("utf-8"))
    except json.JSONDecodeError:
        print("Invalid JSON:", msg.payload)
        return

    cmd    = message.get("cmd_string", "")
    ip_str = message.get("ip_range")

    # 1) auto_start/auto_stop 처리
    if cmd == "auto_start":
        if ip_str == "All" or ip_str == LOCAL_IP:
            auto_event.set()
            print(f"[{datetime.now(korea_tz)}] → 자동 START (ip_range={ip_str})")
        else:
            print(f"[{datetime.now(korea_tz)}] 자동 START 무시 (ip_range={ip_str})")
        return

    if cmd == "auto_stop":
        if ip_str == "All" or ip_str == LOCAL_IP:
            auto_event.clear()
            print(f"[{datetime.now(korea_tz)}] → 자동 STOP (ip_range={ip_str})")
        else:
            print(f"[{datetime.now(korea_tz)}] 자동 STOP 무시 (ip_range={ip_str})")
        return

    # 2) 수동 제어: IP 일치 시만
    if ip_str != LOCAL_IP:
        print(f"[{datetime.now(korea_tz)}] IP 불일치 (받음:{ip_str} 내IP:{LOCAL_IP}) → 무시")
        return

    print(f"[{datetime.now(korea_tz)}] 수동 명령 수신 → {cmd} (ip_range={ip_str})")

    # 수동 명령일 때 manual_event 토글
    if cmd in ("go", "left", "right", "back"):
        manual_cmd = cmd
        manual_event.set()
    elif cmd in ("stop", "mid"):
        # stop 또는 mid 수신 시 정지
        manual_event.clear()
    elif cmd == "exit":
        print("→ exit 수신, 종료 처리")
        exit_event.set()
        client.unsubscribe(COMMAND_TOPIC)
        client.loop_stop()
        client.disconnect()

def main():
    # 제어 루프 스레드 시작
    threading.Thread(target=motion_loop, daemon=True).start()
    
    client = mqtt.Client()
    client.on_connect  = on_connect
    client.on_message  = on_message
    client.on_publish  = on_publish

    try:
        client.connect(BROKER_ADDR, BROKER_PORT)
    except Exception as e:
        print(f"[{datetime.now(korea_tz)}] 초기 연결 실패: {e}")
        return

    client.loop_start()

    try:
        while not exit_event.is_set():
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("KeyboardInterrupt! 종료 처리 중...")
        exit_event.set()
        client.unsubscribe(COMMAND_TOPIC)
        client.loop_stop()
        client.disconnect()
    finally:
        print("프로그램 종료")

if __name__ == "__main__":
    print(f"Local IP: {LOCAL_IP}")
    main()


Local IP: 172.20.10.7
[2025-05-25 06:38:06.652670+09:00] MQTT 연결 성공
[2025-05-25 06:38:06.666359+09:00] 구독 시작 → AGV/command
[2025-05-25 06:38:12.938051+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:13.176635+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:14.469823+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:16.317537+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:16.718545+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:17.969987+09:00] IP 불일치 (받음:None 내IP:172.20.10.7) → 무시
[2025-05-25 06:38:21.845209+09:00] 수동 명령 수신 → go (ip_range=172.20.10.7)
[2025-05-25 06:38:22.769096+09:00] 수동 명령 수신 → stop (ip_range=172.20.10.7)
[2025-05-25 06:38:23.383904+09:00] 수동 명령 수신 → back (ip_range=172.20.10.7)
[2025-05-25 06:38:24.609385+09:00] 수동 명령 수신 → stop (ip_range=172.20.10.7)
[2025-05-25 06:38:25.533495+09:00] 수동 명령 수신 → left (ip_range=172.20.10.7)
[2025-05-25 06:38:26.148452+09:00] 수동 명령 수신 → stop (ip_range=172.20.10.